In [2]:
import os
from bs4 import BeautifulSoup, NavigableString
import pandas as pd

os.chdir('..')
os.getcwd()

'/workspaces/LW15-Functional-Screening'

In [10]:
# Load the CSV file
data = pd.read_csv('TCGA/survival_summary.csv', index_col=0)
data


,gene_name,higher_survival_group
1,SCARF2,Low
2,REEP6,Low
3,GFER,Low
4,B4GALT3,Low
5,SH3BP1,Low
...,...,...
144,PRDX2,High
145,ASAH1,High
146,MYO6,High
147,RANBP2,High


In [11]:
# create survival dict
survival_dict = {}
for index, row in data.iterrows():
    gene_name = row['gene_name']
    survival_dict[gene_name] = row['higher_survival_group']
print(survival_dict)

{'SCARF2': 'Low', 'REEP6': 'Low', 'GFER': 'Low', 'B4GALT3': 'Low', 'SH3BP1': 'Low', 'ARID3C': 'Low', 'RFNG': 'Low', 'NLGN2': 'Low', 'SYCE2': 'Low', 'ZCWPW1': 'Low', 'PRRX2': 'Low', 'LDHD': 'High', 'ARSI': 'Low', 'PLVAP': 'High', 'TRIM62': 'High', 'PTPN6': 'Low', 'PGF': 'Low', 'MYPOP': 'Low', 'HOXC11': 'Low', 'GRIK5': 'Low', 'BRSK1': 'Low', 'CRNDE': 'Low', 'DPF1': 'Low', 'ADAM11': 'Low', 'HOXC5': 'Low', 'BATF3': 'Low', 'CBARP': 'Low', 'MPZL1': 'Low', 'DLGAP3': 'Low', 'MAP4K1': 'Low', 'FNDC4': 'Low', 'PCSK4': 'Low', 'FBXL14': 'High', 'RCOR2': 'Low', 'NCKIPSD': 'High', 'CADM4': 'Low', 'ECD': 'High', 'IRF6': 'High', 'MDFI': 'Low', 'MAFA': 'High', 'WNT5B': 'Low', 'UCK2': 'Low', 'SYNGAP1': 'Low', 'TRNP1': 'Low', 'MLKL': 'Low', 'WNT1': 'Low', 'TUBG2': 'Low', 'DDN': 'Low', 'NTN3': 'Low', 'IRF5': 'Low', 'LTK': 'Low', 'SMARCD1': 'Low', 'GPC2': 'Low', 'FBXL19': 'Low', 'NPR3': 'High', 'ZFTA': 'High', 'CELF5': 'Low', 'PIM2': 'Low', 'CCDC136': 'Low', 'CALCOCO2': 'High', 'AGAP2': 'Low', 'LARGE2': 'Lo

In [13]:

# Load the HTML content
with open('index.html', 'r') as file:
    html_content = file.read()
    
soup = BeautifulSoup(html_content, 'html.parser')

table = soup.find('table', {'id': 'functionalScreening'})

# Add a header cell for the new column if necessary
header_row = table.find('tr')
new_header_cell = soup.new_tag('th')
new_header_cell.string = "P-Value"
header_row.append(new_header_cell)

# Iterate over each row in the table to add a new cell with the p_value
for row in table.find_all('tr')[1:]:  # Skip the header row
    cells = row.find_all('td')
    if not cells:
        continue  # Skip if no cells found
    gene_name = cells[0].text.strip().upper()  # Assuming gene name is in the first cell
    survival = survival_dict.get(gene_name, "")  # Use "" as default if p_value not found
    
    # Create a new cell and append it to the current row
    new_cell = soup.new_tag('td')
    new_cell.string = survival  # Convert p_value to string
    row.append(new_cell)

# Save the modified HTML
with open('updated_index.html', 'w') as file:
    file.write(str(soup))


# TCGA vs LW15 Comparison

For each gene with a non-empty “Expression of Higher Survival group” (13th) column, look in the Regulation in Sensitive (12th) column

if it’s Upregulated & the expression low, it’s same trend, else if upregulated & high it’s dif else if downregulated & low dif, else if downregulated & high same

put this into a new column called TCGA vs LW15 comparison

In [ ]:
# Load the HTML content
with open('index.html', 'r') as file:
    html_content = file.read()
    
soup = BeautifulSoup(html_content, 'html.parser')

table = soup.find('table', {'id': 'functionalScreening'})

# Add a header cell for the new column if necessary
header_row = table.find('tr')
new_header_cell = soup.new_tag('th')
new_header_cell.string = "LW15 vs TCGA"
header_row.append(new_header_cell)

# Iterate over each row in the table to add a new cell with the p_value
for row in table.find_all('tr')[1:]:  # Skip the header row
    cells = row.find_all('td')
    if not cells:
        continue  # Skip if no cells found
    expression = cells[12].text.strip()
    regulation = cells[1].text.strip()

    if expression != "": # check that expression cell isn't empty

        # Low expression cases:
        if regulation == "Upregulated" and expression == "Low":
            trend = "same"
        elif regulation == "Downregulated" and expression == "Low": 
            trend = "dif"

        # High expression cases:
        elif regulation == "Upregulated" and expression == "High": 
            trend = "dif"
        elif regulation == "Downregulated" and expression == "High": 
            trend = "same"
    else: # if the expression cell is empty
        trend = "" 

    # Create a new cell and append it to the current row
    new_cell = soup.new_tag('td')
    new_cell.string = trend  # Convert p_value to string
    row.append(new_cell)

# Save the modified HTML
with open('updated_index.html', 'w') as file:
    file.write(str(soup))
